In [23]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Read data into a dataframe and 
# df = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = df.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]
# densityPlot = sns.kdeplot(counts, bw_adjust=3)
# yLabels = densityPlot.get_yticks()
# densityPlot.set_yticklabels('{:,.0%}'.format(y) for y in yLabels)
# plt.xlabel("Number of reviews")
# plt.ylabel("Percentage of users")
# plt.savefig(fname="densityPlot")
# plt.show()

In [24]:
# import pandas as pd
# import pickle
# # Read ratings into a dataframe and narrow down reviewers to desired review count
# ratings = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = ratings.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]

# # Create and apply mask that removes all other reviewers.
# mask = ratings['reviewerID'].isin(counts.index)
# ratings = ratings[mask]

# # Read metadata into dataframe and define asins that are used by making a set
# metadata = pd.read_csv("metadata.csv")
# used_asins = set(ratings["asin"])

# # Create and apply mask that removes unreferenced items
# mask = metadata["asin"].isin(used_asins)
# metadata = metadata[mask]
# metadata.to_pickle("metadata.pickle")
# ratings.to_pickle("ratings.pickle")
